Read SDC Platinum data

In [68]:
import pandas as pd
from  path_utils import get_base_path
import os

# Set path
base = get_base_path()
relative =  r"05 Analysis\01 Main\00 Python data\05 Deals\SDC gafam - all, combined.xlsx"
file_path = os.path.join(base, relative)

# Load the Excel file
df = pd.read_excel(file_path, header=0)


Read Target assignees file (JSON)

In [ ]:
import json
import os
# Load existing target_assignees_dict from a file if it exists
target_assignees_path = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees.json")

if os.path.exists(target_assignees_path):
    with open(target_assignees_path, "r") as file:
        try:
            target_assignees_dict = json.load(file)
        except json.JSONDecodeError:
            # In case the file is empty or not properly formatted
            target_assignees_dict = {}
else:
    # Initialize an empty dictionary if the file doesn't exist
    target_assignees_dict = {}


In [50]:
import json
import os
from path_utils import get_base_path

# Set path
base = get_base_path()
path_old = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees.json")
path_new = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees_all_2024.11.03_02.34.json")

# Load both JSON files
with open(path_old, 'r') as file:
    old = json.load(file)

with open(path_new, 'r') as file:
    new = json.load(file)

# Combine the two dictionaries
target_assignees_dict_new = {**new, **old}

# Remove duplicates for each target
for target, value in target_assignees_dict.items():
    if 'str_matched' in value and isinstance(value['str_matched'], list):
        # Convert each sublist to a tuple (because lists are not hashable)
        unique_matches = set(tuple(match) for match in value['str_matched'])
        # Convert back to list of lists
        value['str_matched'] = [list(match) for match in unique_matches]

# Save the updated dictionary back to a JSON file
output_path = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees_unique.json")
with open(output_path, "w") as outfile:
    json.dump(target_assignees_dict, outfile, indent=4)

print(f"Updated dictionary with unique matches saved to {output_path}")

Updated dictionary with unique matches saved to C:/Users/amusali/Desktop/uc3m PhD\05 Analysis\01 Main\00 Python data\target_assignees_unique.json


In [59]:
for key, value in target_assignees_dict_new.items():
    if key not in target_assignees_dict:
        print(f"New target '{key}' added to the dictionary.")
        print(value)

New target 'Whisper.AI Inc' added to the dictionary.
{'str_matched': [['WHISPER.AI, Inc.', 'legacy'], ['WHISPER.AI, Inc.', 'legacy'], ['WHISPER.AI, Inc.', 'assignee'], ['WHISPER.AI, Inc.', 'assignee']]}
New target 'Camouflaj' added to the dictionary.
{'str_matched': []}
New target 'BlinkAI Technologies Inc' added to the dictionary.
{'str_matched': [['BlinkAI Technologies, Inc.', 'legacy'], ['BlinkAI Technologies, Inc.', 'legacy'], ['BlinkAI Technologies, Inc.', 'assignee'], ['BlinkAI Technologies, Inc.', 'assignee']]}
New target 'ImagineOptix Corp' added to the dictionary.
{'str_matched': [['IMAGINEOPTIX CORPORATION', 'legacy'], ['IMAGINEOPTIX CORPORATION', 'legacy'], ['IMAGINEOPTIX CORPORATION', 'assignee'], ['IMAGINEOPTIX CORPORATION', 'assignee']]}
New target 'Within Unlimited Inc' added to the dictionary.
{'str_matched': [['Within Unlimited, Inc.', 'legacy'], ['Within Unlimited, Inc.', 'legacy'], ['Within Unlimited, Inc.', 'assignee'], ['Within Unlimited, Inc.', 'assignee']]}
New t

In [61]:
target_assignees_dict['Whisper.AI Inc'] = {'str_matched': [], 'google_matches': []}
target_assignees_dict['Whisper.AI Inc']['str_matched'] = [['WHISPER.AI, Inc.', 'legacy'] ,['WHISPER.AI, Inc.', 'assignee']]

target_assignees_dict['BlinkAI Technologies Inc'] = {'str_matched': [], 'google_matches': []}
target_assignees_dict['BlinkAI Technologies Inc']['str_matched'] = [['BlinkAI Technologies, Inc.', 'legacy'], ['BlinkAI Technologies, Inc.', 'assignee']]

target_assignees_dict['ImagineOptix Corp'] = {'str_matched': [], 'google_matches': []}
target_assignees_dict['ImagineOptix Corp']['str_matched'] = [ ['IMAGINEOPTIX CORPORATION', 'legacy'], ['IMAGINEOPTIX CORPORATION', 'assignee']]


target_assignees_dict['Within Unlimited Inc'] = {'str_matched': [], 'google_matches': []}
target_assignees_dict['Within Unlimited Inc']['str_matched'] = [['Within Unlimited, Inc.', 'legacy'], ['Within Unlimited, Inc.', 'assignee']]



    


In [ ]:
for target, value in target_assignees_dict.items():
    for str_matches, google_matches in value.items():


In [16]:
type(target_assignees_dict['WaveOne Inc']['str_matched'][0][0])
print(set([[2,3], [1,2], [2,3]]))

TypeError: unhashable type: 'list'

Clean to get create a GAFAM deals only 

In [36]:
# Define GAFAM companies
gafam_companies = ["Google", "Microsoft", "Facebook", "Apple", "Amazon"]

# Create GAFAM variable
df['GAFAM'] = df['Acquiror Full Name'].apply(lambda x: 1 if any(company.lower() in x.lower() for company in gafam_companies) else 0)

# Get TargetFullName for GAFAM == 1
gafam_deals = df[(df['GAFAM'] == 1) & (df['Acquiror Full Name'] != df['Target Full Name'])]['Target Full Name']
gafam_deals = gafam_deals

# Initialize an empty dictionary to store the results
target_assignees_dict = {}


In [54]:
for key, value in target_assignees_dict.items():
    print(key, value)

   # if 'google_matches' in value and isinstance(value['google_matches'], list):
        # Convert each sublist to a tuple (because lists are not hashable)
     #   unique_matches = set(tuple(match) for match in value['google_matches'])
        # Convert back to list of lists
       # value['google_matches'] = [list(match) for match in unique_matches]

Fox Software Inc {'str_matched': [], 'google_matched': []}
Stac Electronics Inc {'str_matched': [['Stac Electronics, Inc.', 'legacy']], 'google_matched': ['Stac Electronics, Inc.']}
Altamira Software Corp {'str_matched': [], 'google_matched': []}
One Tree Software {'str_matched': [], 'google_matched': []}
UUNet Technologies Inc {'str_matched': [['UUNet Technologies, Inc.', 'legacy']], 'google_matched': ['UUNet Technologies, Inc.']}
Wang Laboratories Inc {'str_matched': [['Wang Laboratories, Inc.', 'legacy']], 'google_matched': ['Wang Laboratories, Inc.']}
Individual Inc {'str_matched': [['Individual, Inc.', 'legacy'], ['INDIVIDUAL MONITORING SYSTEMS, INC', 'legacy'], ['Individual Implant Solutions, Inc.', 'legacy'], ['Individual Software, Inc.', 'legacy'], ['Individual Network, LLC', 'legacy']], 'google_matched': ['Individual, Inc.', 'Individual Software, Inc.', 'Individual Network, LLC']}
Blue Ribbon SoundWorks Ltd {'str_matched': [], 'google_matched': []}
Netwise Inc {'str_matched': 

Get best matches from PatensView API

In [18]:
target_assignees_dict = {}

In [11]:
import api.get_closest_assignees as gca
import importlib
importlib.reload(gca)

import re

# Total number of targets (counting those already in the dictionary)
targets = df['TargetFullName'].tolist()
total_targets = len(df)

# Initialize the counter for processed targets
processed_count = len(target_assignees_dict)


# Iterate over each target and apply the get_closest_assignees function
for i, target in enumerate(targets):
    # Clean target name by removing anything in parentheses (including parentheses)
    cleaned_target = re.sub(r"\(.*?\)", "", target).strip()
    cleaned_target = re.sub(r"\{.*?\}", "", cleaned_target).strip()

    # Check if the cleaned target already exists in the saved dictionary
    if cleaned_target in old:
        #print(f"Skipping {cleaned_target} as it already exists in the saved file.")
        processed_count += 1
        continue
     
    print(f"looking for {cleaned_target}...")
    # Get the closest assignees for the cleaned target
    best_matches = gca.get_closest_assignees(cleaned_target, threshold=50)
    
    # Store the result in the dictionary
    old[cleaned_target] = {}
    old[cleaned_target]['str_matched'] = best_matches if best_matches else []
    
    # Update the processed count
    processed_count += 1

    # Print progress every 25 targets
    if processed_count % 25 == 0:
        print(f"Have completed {processed_count}/{total_targets} targets.")


looking for Whisper.AI Inc...
{'assignees': [{'assignee_id': 'd21599dd-d075-465a-a4a8-40a7890c803f', 'assignee_lastknown_country': 'US', 'assignee_organization': 'WHISPER.AI, Inc.'}], 'count': 1, 'total_assignee_count': 1}
Adding ('WHISPER.AI, Inc.', 'legacy')  from the source:  legacy
whisper.ai  is being matched to [(['WHISPER.AI, Inc.'], 90)]
{'assignees': [{'assignee_id': 'd21599dd-d075-465a-a4a8-40a7890c803f', 'assignee_lastknown_country': 'US', 'assignee_organization': 'WHISPER.AI, Inc.'}], 'count': 1, 'total_assignee_count': 1}
Adding ('WHISPER.AI, Inc.', 'legacy')  from the source:  legacy
whisper.ai  is being matched to [(['WHISPER.AI, Inc.'], 90)]
{'error': False, 'count': 1, 'total_hits': 1, 'assignees': [{'assignee_id': 'b4e89f06-7d78-4c0f-ba46-d4f85996e120', 'assignee_organization': 'WHISPER.AI, Inc.', 'assignee_lastknown_country': 'US'}]}
Adding ('WHISPER.AI, Inc.', 'assignee')  from the source:  assignee
whisper.ai  is being matched to [(['WHISPER.AI, Inc.'], 90)]
{'erro

Save the best matches as dict.

In [ ]:
# Save the updated target_assignees_dict back to the JSON file
target_assignees_path = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees_all_2024.11.03_02.34.json")

with open(target_assignees_path, "w") as outfile:
    json.dump(target_assignees_dict, outfile, indent=4)

: 

In [19]:
empty_match_count = 0
for key, value in target_assignees_dict.items():
    if 'str_matched' in value and isinstance(value['str_matched'], list) and value['str_matched']:
        empty_match_count += 1

print(f"Number of keys where 'str_matched' is an empty list: {empty_match_count}")


Number of keys where 'str_matched' is an empty list: 201


## Update the treated assignees to the Excel file


In [71]:
import json
import pandas as pd
import os
from path_utils import get_base_path

mydict = target_assignees_dict
# Collect the data: target company names and matched assignee names
data_to_append = []

for target, value in mydict.items():

    str_matches = value['str_matched']
    for match_tuple in str_matches:
        assignee_name = match_tuple[0]  # Get the first item of the tuple (the matched assignee name)
        source = match_tuple[1]  # Get the second item of the tuple (the source of the match)
        data_to_append.append({'Target Company': target, 'Assignees': assignee_name, 'Source': source, 'GoogleChecked': 0})

# Create a DataFrame from the collected data
df_to_append = pd.DataFrame(data_to_append)

merged_df = pd.merge(df, df_to_append, left_on='TargetFullName', right_on='Target Company', how='left')

print(len(df_to_append))
print(len(df))
print(len(merged_df))


# Save the updated DataFrame to the Excel file

base = get_base_path()
excel_file_path = os.path.join(base, r"05 Analysis\01 Main\00 Python data\05 Deals\Master - all.xlsx")

merged_df.to_excel(excel_file_path, index=False)


352
502
834


Save **treated** assignees

In [ ]:
import json
import pandas as pd
import os
from path_utils import get_base_path
# File paths
excel_file_path = os.path.join(get_base_path(),r"05 Analysis\01 Main\api\treated_assignees-all_matches.xlsx")

# Load the JSON data
if  os.path.exists(excel_file_path):
    with open(target_assignees_path, "r") as file:
        target_assignees_dict = json.load(file)
else:
    

# Collect the data: target company names and matched assignee names
data_to_append = []

for target, value in target_assignees_dict.items():
    # Ensure 'str_matched' exists and is a list of tuples
    if 'str_matched' in value and isinstance(value['str_matched'], list):
        matches = value['str_matched']
        for match_tuple in matches:
            assignee_name = match_tuple[0]  # Get the first item of the tuple (the matched assignee name)
            data_to_append.append({'Target Company': target, 'Assignees': assignee_name})

# Create a DataFrame from the collected data
df_to_append = pd.DataFrame(data_to_append)

# Check if the Excel file already exists
if os.path.exists(excel_file_path):
    # Load existing data
    existing_df = pd.read_excel(excel_file_path)
    # Concatenate the new data with the existing data
    updated_df = pd.concat([existing_df, df_to_append], ignore_index=True)
else:
    # If the file doesn't exist, the updated data is just the new data
    updated_df = df_to_append

# Save the updated DataFrame to the Excel file
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='w') as writer:
    updated_df.to_excel(writer, index=False, sheet_name='Sheet1')

print("Data has been successfully appended to 'treated_assignees.xlsx'.")

Data has been successfully appended to 'treated_assignees.xlsx'.


**Double check the matches with Google Search**

In [21]:
totalcount = 0
for key, value in target_assignees_dict.items():
    if 'str_matched' in value and isinstance(value['str_matched'], list) and value['str_matched']:
        #print(value['str_matched'])
        totalcount += 1

print(f"Number of keys where 'str_matched' is an non-empty list: {totalcount}")

Number of keys where 'str_matched' is an non-empty list: 201


In [64]:
import api.google_search_checker as gsc
from api.google_search_checker import perform_google_search
import importlib

importlib.reload(gsc)

gsc.perform_google_search('Motorola Mobility d Inc', ['Motorola Mobility LLC'])

<Response [429]>
[]
It took 0.24 secs to find the successfull match.


([], 2)

In [23]:
from api.google_search_checker import perform_google_search

local_counter = 0

## Run over the target assignees dict to run the Google Search
for key, value in target_assignees_dict.items():

    ## Skip the loop if no match exists
    if not value['str_matched']:
        continue
    
    ## Get the name  of the target 
    target = key

    ## Get the name of the matches names in a list
    if 'google_matched' not in value:
        value['google_matched'] = []
    matched_names = [x[0] for x in value['str_matched'] if len(value['google_matched']) == 0]
    
    ## Perform the google search
    google_matches, apicallcount = perform_google_search(target, matched_names)

    ## Assign back to the original dictionary
    value['google_matched'] = google_matches

    ## Print progress
    local_counter += 1
    print(f"Google search progress : {local_counter/totalcount}")

    ## Print API counts
    if apicallcount % 50 == 0:
        print("API call count is: ", apicallcount)



It took 0.71 secs to find the successfull match.
Google search progress : 0.004975124378109453
A Saved API
It took 0.21 secs to find the successfull match.
Google search progress : 0.009950248756218905
A Saved API
It took 0.23 secs to find the successfull match.
Google search progress : 0.014925373134328358
A Saved API
It took 0.36 secs to find the successfull match.
Google search progress : 0.01990049751243781
A Saved API
It took 0.45 secs to find the successfull match.
Google search progress : 0.024875621890547265
A Saved API
It took 0.21 secs to find the successfull match.
Google search progress : 0.029850746268656716
A Saved API
It took 0.19 secs to find the successfull match.
Google search progress : 0.03482587064676617
A Saved API
It took 0.28 secs to find the successfull match.
Google search progress : 0.03980099502487562
A Saved API
It took 0.24 secs to find the successfull match.
Google search progress : 0.04477611940298507
A Saved API
It took 0.27 secs to find the successfull

**save to an excel**

In [30]:
import json
import pandas as pd
import os
from path_utils import get_base_path
# File paths
excel_file_path = os.path.join(get_base_path(),r"05 Analysis\01 Main\00 Python data\True Matches by Google.xlsx")

# Load the JSON data
with open(target_assignees_path, "r") as file:
    target_assignees_dict = json.load(file)

# Collect the data: target company names and matched assignee names
data_to_append = []

for target, value in target_assignees_dict.items():
    # Ensure 'str_matched' exists and is a list of tuples
    if 'google_matched' in value and isinstance(value['google_matched'], list):
        matches = value['google_matched']

        for match in matches:
            data_to_append.append({'Target Company': target, 'Assignees': match, 'str_matches': value['str_matched']})

# Create a DataFrame from the collected data
df_to_append = pd.DataFrame(data_to_append)

# Check if the Excel file already exists
if os.path.exists(excel_file_path):
    # Load existing data
    existing_df = pd.read_excel(excel_file_path)
    # Concatenate the new data with the existing data
    updated_df = pd.concat([existing_df, df_to_append], ignore_index=True)
else:
    # If the file doesn't exist, the updated data is just the new data
    updated_df = df_to_append

# Save the updated DataFrame to the Excel file
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='w') as writer:
    updated_df.to_excel(writer, index=False, sheet_name='cleaned_all')

print("Data has been successfully appended to 'treated_assignees.xlsx'.")

Data has been successfully appended to 'treated_assignees.xlsx'.
